<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-моделей" data-toc-modified-id="Исследование-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование моделей</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [29]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.dummy import DummyClassifier

data = pd.read_csv('/datasets/Churn.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [2]:
data.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


Убедимся, что нет дубликатов:

In [3]:
data.duplicated().sum()

0

Убедимся, что нет дубликатов среди `CustomerId`

In [4]:
data['CustomerId'].duplicated().sum()

0

Заметим, что фамилия, номер строки, ид клиента нам никак не должны помочь (очевидно, что эти данные никак не связаны с тем, уходит клиент или нет), удалим этот столбецы

In [5]:
data = data.drop(['Surname', 'RowNumber', 'CustomerId'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           9091 non-null   float64
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


Посмотрим на страны, которыми представлены клиенты:

In [6]:
data['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

Убедимся, что пол представлен только двумя значениями:

In [7]:
data['Gender'].unique()

array(['Female', 'Male'], dtype=object)

Посмотрим уникальные значения `Tenure`

In [8]:
data['Tenure'].unique()

array([ 2.,  1.,  8.,  7.,  4.,  6.,  3., 10.,  5.,  9.,  0., nan])

In [9]:
data[data['Tenure'].isna()]

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
30,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
...,...,...,...,...,...,...,...,...,...,...,...
9944,744,Germany,Male,41,NaN,190409.34,2,1,1,138361.48,0
9956,520,France,Female,46,NaN,85216.61,1,1,0,117369.52,1
9964,479,France,Male,34,NaN,117593.48,2,0,0,113308.29,0
9985,659,France,Male,36,NaN,123841.49,2,1,0,96833.00,0


У нас около 9% клиентов без значения `Tenure`. Причины не ясны, по значениям видно, что есть значения от 0 до 10 включая, а значит заполнить какими-то данными мы не можем (например 0), оставим пока что как есть

Проверим значения продуктов банка, изпользуемых клиентами:

In [10]:
data['NumOfProducts'].unique()

array([1, 3, 2, 4])

А также булевы значения: `IsActiveMember`, `HasCrCard`, `Exited`

In [11]:
data['HasCrCard'].unique()

array([1, 0])

In [12]:
data['IsActiveMember'].unique()

array([1, 0])

In [13]:
data['Exited'].unique()

array([1, 0])

**Вывод:** перед нами датасет с 10 000 записями, дубликатов нет, неявных дубликатов тоже нет. Удалили столбецы с фамилией, номером строки и ид, они нам не помогут. Пользователи из трех стран: Франция, Испания, Герамания. Столбец `Tenure` содержит пропуски, заполнить их не представляется возможным.

Примемим технику прямого кодирования для столбцов: `Geography`, `Gender`, `Tenure`, `NumOfProducts`

In [14]:
data = pd.get_dummies(data, drop_first=True, columns=['Geography', 'Gender', 'Tenure', 'NumOfProducts'])

Разделим датасет на 3 части: обучающая, валидационная, тестовая - в процентом отношении 60%, 20% и 20% соответственно

In [15]:
features = data.drop('Exited', axis=1)
target = data['Exited']

features_train, features_valid, target_train, target_valid = train_test_split(
    features, 
    target, 
    test_size=0.4, 
    random_state=42
)
    
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, 
    target_valid, 
    test_size=0.5, 
    random_state=42
)

In [16]:
display(features_train.shape)
display(features_valid.shape)
features_test.shape

(6000, 22)

(2000, 22)

(2000, 22)

**Вывод:** дубликотов в исходном массиве данных нет; столбцы фамилия, номер строки и ид клиента мы удалили, потому что они никак не помогают обучению. Применили технику прямого кодирования к столбцам: Geography, Gender, Tenure, NumOfProducts. Разбили исходный датасет на 3 части: обучающая, валидационная, тестовая - в процентом отношении 60%, 20% и 20% соответственно

## Исследование моделей

Обучим решающее дерево с разным набором параметров.

In [17]:
def get_best_decision_tree_classifier(features_train, target_train, features_valid, target_valid):
    best_model = None
    best_result = 0
    for depth in range(1, 21):
        for min_samples_leaf in range(1, 10):
            model = DecisionTreeClassifier(
                random_state=42, 
                max_depth=depth, 
                min_samples_leaf=min_samples_leaf
            )
            model.fit(features_train, target_train) 
            predictions_valid = model.predict(features_valid) 
            result = f1_score(target_valid, predictions_valid)
            if result > best_result:
                best_result = result
                best_model = model
                
    return best_model

model = get_best_decision_tree_classifier(features_train, target_train, features_valid, target_valid)
predictions_valid = model.predict(features_valid) 
f1_score(target_valid, predictions_valid)

0.5266666666666666

**Вывод:** для решающего дерева глубиной 7 с критерием entropy, мы получили лучший результат **f1: 0.56**

In [18]:
def get_roc_auc_score(model, features, target):
    probabilities = model.predict_proba(features)
    return roc_auc_score(target, probabilities[:, 1])

get_roc_auc_score(model, features_valid, target_valid)

0.8087857374918778

**Вывод:** для решающего дерева глубиной 7 с критерием entropy, мы получили **roc_auc_score: 0.81**

Обучим случайный лес с разным набором параметров:

In [19]:
def get_best_random_forest_classifier(features_train, target_train, features_valid, target_valid):
    best_model = None
    best_result = 0
          
    for max_depth in range(1, 21):    
        for n_estimators in range(20, 125, 1):
            model = RandomForestClassifier(
                random_state=42, 
                n_estimators=n_estimators,
                max_depth=max_depth
            ) 
            model.fit(features_train, target_train)
            predictions_valid = model.predict(features_valid) 
            result = f1_score(target_valid, predictions_valid)
            if result > best_result:
                best_result = result
                best_model = model
                
    return best_model
    
model = get_best_random_forest_classifier(features_train, target_train, features_valid, target_valid)
predictions_valid = model.predict(features_valid) 
f1_score(target_valid, predictions_valid)

0.570957095709571

**Вывод:** для случайного леса мы получили лучший результат **f1: 0.56**

In [20]:
get_roc_auc_score(model, features_valid, target_valid)

0.8371093242365171

**Вывод:** для случайного леса мы получили **roc_auc_score: 0.83**

## Борьба с дисбалансом

In [21]:
data['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [22]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=42)
    
    return features_upsampled, target_upsampled

def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=42)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=42)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=42)
    
    return features_downsampled, target_downsampled

In [23]:
best_models = []
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

model = get_best_decision_tree_classifier(features_upsampled, target_upsampled, features_valid, target_valid)
predictions_valid = model.predict(features_valid) 
result = f1_score(target_valid, predictions_valid)
        
best_models.append(model)
roc_auc = get_roc_auc_score(model, features_valid, target_valid)
model.get_params(), result, roc_auc

({'ccp_alpha': 0.0,
  'class_weight': None,
  'criterion': 'gini',
  'max_depth': 7,
  'max_features': None,
  'max_leaf_nodes': None,
  'min_impurity_decrease': 0.0,
  'min_impurity_split': None,
  'min_samples_leaf': 7,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'random_state': 42,
  'splitter': 'best'},
 0.522837706511176,
 0.8038084795321638)

При увеличении выборки мы не добились результата, лучшее решающее дерево с максимальной глубиной 7 и результатом 0,52. AUC-ROC = 0.80

In [24]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

model = get_best_decision_tree_classifier(features_downsampled, target_downsampled, features_valid, target_valid)
predictions_valid = model.predict(features_valid) 
result = f1_score(target_valid, predictions_valid)
        
best_models.append(model)
roc_auc = get_roc_auc_score(model, features_valid, target_valid)
model.get_params(), result, roc_auc

({'ccp_alpha': 0.0,
  'class_weight': None,
  'criterion': 'gini',
  'max_depth': 7,
  'max_features': None,
  'max_leaf_nodes': None,
  'min_impurity_decrease': 0.0,
  'min_impurity_split': None,
  'min_samples_leaf': 6,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'random_state': 42,
  'splitter': 'best'},
 0.5229885057471264,
 0.805921052631579)

Можем заметить, что если мы уменьшим выборку тех, клиентов, которые остались (оставив только 25% от первоначального числа), то для решающего дерева мы получим лучший результат 0.52, что так же хуже результута без всяких изменений выборки. AUC-ROC = 0.80

In [25]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

model = get_best_random_forest_classifier(features_upsampled, target_upsampled, features_valid, target_valid)
predictions_valid = model.predict(features_valid) 
result = f1_score(target_valid, predictions_valid)
        
best_models.append(model)
roc_auc = get_roc_auc_score(model, features_valid, target_valid)
model.get_params(), result, roc_auc

({'bootstrap': True,
  'ccp_alpha': 0.0,
  'class_weight': None,
  'criterion': 'gini',
  'max_depth': 17,
  'max_features': 'auto',
  'max_leaf_nodes': None,
  'max_samples': None,
  'min_impurity_decrease': 0.0,
  'min_impurity_split': None,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'n_estimators': 40,
  'n_jobs': None,
  'oob_score': False,
  'random_state': 42,
  'verbose': 0,
  'warm_start': False},
 0.5970937912813739,
 0.8402038661468486)

Видим, что удалось достигнуть наилучшей оценки в 0.597 для случайного леса с 40 n_estimators и max_depth = 17. "Апскейлить" пришлось в 4 раза. AUC-ROC = 0.84

In [26]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

model = get_best_random_forest_classifier(features_downsampled, target_downsampled, features_valid, target_valid)
predictions_valid = model.predict(features_valid) 
result = f1_score(target_valid, predictions_valid)
        
best_models.append(model)
roc_auc = get_roc_auc_score(model, features_valid, target_valid)
model.get_params(), result, roc_auc

({'bootstrap': True,
  'ccp_alpha': 0.0,
  'class_weight': None,
  'criterion': 'gini',
  'max_depth': 19,
  'max_features': 'auto',
  'max_leaf_nodes': None,
  'max_samples': None,
  'min_impurity_decrease': 0.0,
  'min_impurity_split': None,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'n_estimators': 95,
  'n_jobs': None,
  'oob_score': False,
  'random_state': 42,
  'verbose': 0,
  'warm_start': False},
 0.555229716520039,
 0.8341804743339831)

Можем заметить, что если мы уменьшим выборку тех, клиентов, которые остались (оставив только 25% от первоначального числа), то для случайного леса мы получим наилучший результат 0.555, что даже выше, чем требовалось. А вот AUC-ROC = 0.838, что ниже, чем у модели выше

Вывод: мы использовали два метода для борьбы с дисбалансом: увеличение выборки и уменьшение выборки

## Тестирование модели

Протестируем все лучшие модели, выведем их параметры и оценки

In [27]:
for model in best_models:
    predictions_test = model.predict(features_test) 
    result = f1_score(target_test, predictions_test)
    roc_auc = get_roc_auc_score(model, features_test, target_test)
    display(model.get_params(), result, roc_auc)

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 7,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 7,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 42,
 'splitter': 'best'}

0.594642857142857

0.8245571026514589

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 7,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 6,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 42,
 'splitter': 'best'}

0.5691489361702128

...

0.6269368295589988

0.8643956450896163

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 19,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 95,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

0.6106594399277325

0.858437268552807

Вывод: выяснилось, что все модели показали результат выше нужного, но лучше всех справился случайный лес с максимальной глубиной 17, и n_estimators=40, обученной на уменьшенной выборке в 50%. Его результат 0.626. Лучший AUC-ROC = 0.864 у этой же модели

In [32]:
dummy_model = DummyClassifier(strategy='constant', constant=1)
dummy_model.fit(features_train, target_train)
f1_score(dummy_model.predict(features_test), target_test)

0.35390946502057613

**Вывод:** на дамми-модели мы получаем f1_score = 0.35, что заметно ниже наших лучших оценок. Проверку на адекватность наши модели прошли

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*

## Общий вывод

**Основные этапы исследования об оттоке клиентов Бета-Банка**
1. Был выгружен один датасет с пользовательскими данными и тарифами: 
    
    - `RowNumber` — индекс строки в данных
    - `CustomerId` — уникальный идентификатор клиента
    - `Surname` — фамилия
    - `CreditScore` — кредитный рейтинг
    - `Geography` — страна проживания
    - `Gender` — пол
    - `Age` — возраст
    - `Tenure` — сколько лет человек является клиентом банка
    - `Balance` — баланс на счёте
    - `NumOfProducts` — количество продуктов банка, используемых клиентом
    - `HasCrCard` — наличие кредитной карты
    - `IsActiveMember` — активность клиента
    - `EstimatedSalary` — предполагаемая зарплата
    - `Exited` — факт ухода клиента (целевой признак)
   
2. В ходе работы разбили исходный датасет на 3 выборки:
    - обучающая
    - валидационная
    - тестовая
    
    В процентном соотношении 60%, 20% и 20% соответственно
3. Исследование модели
    1. Решающее дерево
        Обучили решающие деревья с различными параметрами и `random_state=42`
        Решающее дерево глубиной 7 подошло лучше всего с f1=0,56 и ROC-AUC = 0.81
    2. Случайный лес
        Обучили несколько моделей с различными параметрами. На валидационной выборке, лучший результат f1=0,56 и ROC-AUC = 0.837
    
4. Борьба с дисбалансом
    1. Сначала мы "апскеили" выборку клиентов, которые уходили из банка. Для решающего дерева и случайного леса мы увеличили выборку в 4 раза и обучали модели, лучшие добавили в отдельный список лучших
    2. Далее мы случайным образом удаляли из выборки часть оставшихся клиентов, и обучали модели, удаляли 75%. Лучшие добавили в отдельный список лучших
    
5. Тетсирование моделей

    Наилучий результат показала модель случайного леса с максимальной глубиной 17, и n_estimators=40, обученной на увеличенной выборке в 4 раза. Его результат f1 = 0.626. Лучший AUC-ROC у него же и равняется 0.866